# Lab 11. Inne przykłady wykresów `plotly` oraz analiza głównych składowych

## 1. Kilka innych przykładów wykresów z biblioteką `plotly`

In [12]:
# zainstaluj, jeżeli brakuje biblioteki
import plotly as py
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets

In [13]:
# some more libraries to plot graph
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot


init_notebook_mode(connected = True)

data = dict(type = 'choropleth',

            # kody krajów: https://country.io/iso3.json
            locations = ['POL', 'DEU', 'FRA'],

            # Europa
            locationmode = 'ISO-3',

            # skala kolorów
            colorscale = 'Greens',

            # tekst dodatkowo wyświetlany obok wartości po najechaniu na obszar
            text = ['text 1', 'text 2', 'text 3'],
            # faktyczny wektor wartości, który reprezentuje poszczególne kraje
            z = [1.0, 2.0, 3.0],
            colorbar = {'title': 'Skala wartości'})

layout = dict(geo ={'scope': 'europe'})

choromap = go.Figure(data = [data], layout = layout)

iplot(choromap)

Inne wykresy oparte o otwarte mapy dostępne są pod linkiem: https://plotly.com/python/tile-county-choropleth/

### 1.1 Przykład wykresów interaktywnych

> **UWAGA!**  
> Poniższe przykłady ilustrują możliwe wykorzystanie iPyWidgets oraz plotly do tworzenia interaktywnych wykresów w Jupyter Notebook,
> jednak nie zawsze działają poprawnie z przyczyn póki co niewyjaśnionych.
>
> Lista kontrolek: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#date-picker

In [14]:
# wczytujemy zbiór danych
df = pd.read_csv('../lab_10/data/zamowienia.csv', sep=';')
# konwersja kolumny z datą
df['Data zamowienia'] = pd.to_datetime(df['Data zamowienia'])
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../lab_10/data/zamowienia.csv'

In [ ]:
df.info()

In [ ]:


kraj = widgets.Dropdown(
    options=list(df['Kraj'].unique()),
    value=df['Kraj'][0],
    description='Kraj:',
)


sprzedawca = widgets.Dropdown(
    description='Sprzedawca:   ',
    value=df[df['Kraj'] == kraj.value]['Sprzedawca'][0],
    options=df[df['Kraj'] == kraj.value]['Sprzedawca'].unique().tolist()
)


trace2 = go.Histogram(x=df['Utarg'], opacity=0.75, name='Rozkład utargu')
g1 = go.FigureWidget(data=[trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Sprzedaż'
                        ),
                        barmode='overlay'
                    ))

In [ ]:
def validate():
    if kraj.value in df['Kraj'].unique() and sprzedawca.value in df['Sprzedawca'].unique():
        return True
    else:
        return False


def response(change):
    # update values for sprzedawca
    sprzedawca.options = df[df['Kraj'] == kraj.value]['Sprzedawca'].unique().tolist()
    if validate():
        data = df[(df['Sprzedawca'] == sprzedawca.value) & (df['Kraj'] == kraj.value)]
        with g1.batch_update():
            g1.data[0].x = data['Utarg']
            # g.layout.xaxis = 'Etykieta'
            # g.layour.yaxis = 'Etykieta'

# dodanie nasłuchiwania zmiany wartości w kontrolkach
kraj.observe(response, names="value")
sprzedawca.observe(response, names="value")

In [ ]:
g1

In [ ]:
container2 = widgets.HBox([kraj, sprzedawca])
widgets.VBox([container2, g1])

In [ ]:
# dane, które zostaną przedstawione na wykresie to suma skumulowana wartości zamowienia po dacie
plot_data = df[['Kraj', 'Data zamowienia','Utarg']].groupby(['Kraj', 'Data zamowienia']).agg({'Utarg': 'sum'}).cumsum()
plot_data = plot_data.reset_index()
plot_data

In [ ]:
# deklaracja widgetów

kraj = widgets.Dropdown(
    options=list(plot_data['Kraj'].unique()),
    value=plot_data['Kraj'][0],
    description='Kraj:',
)

# deklaracja wykresu
fig = go.Scatter(x=plot_data['Utarg'], name='Skumulowany utarg', mode='lines')

g2 = go.FigureWidget(data=[fig],
                    layout=go.Layout(
                        title=dict(
                            text='Sprzedaż skumulowana'
                        ),
                        barmode='overlay'
                    ))

# funkcja walidująca poprawność danych w kontrolkach
def validate():
    if kraj.value in plot_data['Kraj'].unique():
        return True
    else:
        return False

# funkja uruchamiana przy zmianie stanu kontrolki (aktualizuje wykres)
def response(change):
    if validate():
        dat = plot_data[plot_data['Kraj'] == kraj.value]
        with g2.batch_update():
            g2.data[0].x = dat['Utarg']
            # g.layout.xaxis = 'Data'
            # g.layour.yaxis = 'Wartość zamówienia'


kraj.observe(response, names="value")

In [ ]:
g2

In [ ]:
container = widgets.HBox([kraj])
widgets.VBox([container, g2])

## 2. Analiza głównych składowych (PCA)

In [ ]:
!pip install scikit-learn

In [ ]:
# wykorzystanie przykładów redukcji wymiarowości z wykładu

# dodanie wizualizacji wybranych cech, aby pokazać ich rozkład pod analizę
# ale może i aby dokonać wyboru mechanizmu redukcji wymiarowości

# policzenie wariancji, aby to również wskazać

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, KernelPCA

In [ ]:
# wczytujemy zbiór danych iris

iris = load_iris()
feature_names = iris.feature_names
feature_names

**Najpierw przeprowadzimy liniową redukcję wymiarowości**

In [ ]:
pca = PCA(n_components=2)        # redukcja do 2 wymiarów
reduced = pca.fit_transform(iris.data)

In [ ]:
# oryginalny shape (rows, columns/features) vs. po redukcji wymiarów
iris.data.shape, reduced.shape

In [ ]:
# jak wygląda wyliczona wariancja dla zwróconych cech
pca.explained_variance_ratio_

In [ ]:
# sumarycznie wyjdzie nam, że te dwie cechy opisują niemal 98% zmienności z tych danych
sum(pca.explained_variance_ratio_)

In [ ]:
# jak wygląda nowy zbiór cech
pca.get_feature_names_out()

**Inny przykład**

In [ ]:
# scalamy dane ze zbioru iris z scikit learn w ramkę danych
iris_df = pd.DataFrame(np.hstack([iris.data, np.expand_dims(iris.target, axis=1)]), columns=iris.feature_names + ['target'])

In [ ]:
iris_df.head()

In [ ]:
pca = PCA(n_components=.99)        # redukcja do poziomu minimum 99% zachowanej wariancji
# pca.feature_names_in_ = iris_df.columns[:-1]
reduced = pca.fit_transform(iris_df[iris_df.columns[:-1]], iris_df[iris_df.columns[-1]])

In [ ]:
# oryginalny shape (rows, columns/features) vs. po redukcji wymiarów
iris.data.shape, reduced.shape

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# sumarycznie wyjdzie nam, że te dwie cechy opisują niemal 99.5% zmienności z tych danych
sum(pca.explained_variance_ratio_)

In [ ]:
# nazwy cech w zbiorze wynikowym (ale to nam niewiele mówi)
pca.get_feature_names_out()

In [ ]:
# liczba cech wejściowych
pca.n_features_in_

In [ ]:
# składowe każdego nowego wymiaru zbioru (n_components × n_features_in)
pca.components_

In [ ]:
# sumaryczna waga każdej cechy
np.sum(np.abs(pca.components_), axis=0)

Wychodzi na to, że cecha `petal width` charakteryzuje się najmniejszą wagą i powinna zostać odrzucona jako pierwsza. Kolejna byłaby cecha `petal length` itd.

Możemy to również policzyć z danych wyliczonych przez algorytm PCA.

In [ ]:
# sortujemy cechy od najważniejszych do najmniej ważnych w kontekście miary wykorzystanej przez algorytm PCA
features_order = np.argsort(np.sum(np.abs(pca.components_), axis=0))[::-1]

In [ ]:
pca.feature_names_in_[features_order].tolist()

OK, widzimy, że PCA zredukowało wymiar cech w zbiorze automatycznie na podstawie wariancji odrzucając jedną cechę. Redukcja liczby cech następuje tak długo, póki usunięcie kolejnej cechy nie zejdzie poniżej założonego progu sumarycznej wariancji. Ale jak wygląda sprawa z intuicją liniowej redukcji wymiarów?

Popatrzmy na wykres poniżej, który prezentuje macierz rozkładu wartości klasy decyzyjnej (species, u nas target) dla każdej cechy zbioru. Jeżeli znajdziemy takie cechy, których punkty dla klasy decyzyjnej nie nachodzą na siebie gdybyśmy je rzutowali na linię (bo tu mamy 2 wymiary, rzutujemy na 1 wymiar), co oznacza, że są separowalne liniowo, to są to algorytm liniowy PCA na pewno będzie wybierał je jako pierwsze do odrzucenia.

In [ ]:
import plotly.express as px

df = px.data.iris()
features = ["sepal_width", "sepal_length", "petal_width", "petal_length"]

fig = px.scatter_matrix(
    df,
    dimensions=features,
    color="species"
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
# ten sam (niemal) wykres, ale za pomocą biblioteki seaborn
import seaborn as sns

# Rename classes using the iris target names
_ = sns.pairplot(iris_df, hue="target")

**Wyświetlamy ponownie taki sam typ wykresu dla nowych cech zbioru po redukcji wymiarowości**

In [ ]:
reduced_df = pd.DataFrame(np.hstack([reduced, np.expand_dims(iris_df.iloc[:, -1], axis=1)]))
reduced_df.head()

In [ ]:
_ = sns.pairplot(reduced_df, hue=3)

## Zadania

**Zadanie 1**

Wykorzystując dowolny zbiór danych ze zbioru EUROSTAT, wyświetl jedną z cech numerycznych w postaci wykresu z mapą z przykładu w tym labie. Musisz dokonać mapowania nazw krajów, gdyż w parametrach wykresu oraz danych EUROSTAT gdyż te nazwy się trochę różnią. Link do pliku, który to ułatwi jest w komentarzu w kodzie przykładu wykresu z mapą.

**Zadanie 2**

Wykorzystując zbiór wine z biblioteki sklearn (`load_wine()`), podobnie jak ze zbiorze iris) dokonaj analizy PCA tak aby:
* zachować minimum 99% wariancji
* zachować minimum 95% wariancji
* zachować minimum 90% wariancji

Ile cech pozostało w tych zbiorach?

**Zadanie 3**  

Wykorzystując przykład w labie wygeneruj wykres korelacji wartości cech wględem klasy decyzyjnej (target) pokazany w labie (scatter plot). Czy cechy, które PCA odrzuciło faktycznie były wskazane jako najlepsi kandydaci pod względem separowalności liniowej?

**Zadanie 4**  

Wykorzystując przykład w skrypcie z wykładu numer 11 (https://github.com/agazbrzezny/MAD_SWPS_2025/blob/master/wyklad_10/redukcja_wielowym.ipynb) pokazujący użycie `SelectFromModel` (punkt 4 wykładu) wykonaj tę samą sekwencję czynności z użyciem klasyfikatora RandomForest do selekcji cech. Po przetrenowaniu modelu drzewa stwórz 3 zbiory cech z progiem jak w zadaniu 2 sumując `feature importance`. Czy zostały wybrane te same cechy co w zadaniu 2?

In [21]:
!pip install eurostat
import eurostat
import json
import requests
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from ipywidgets import widgets
from google.colab import output
output.enable_custom_widget_manager()
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [22]:
#zad 1
data = eurostat.get_data_df('ilc_di03')
data = data.rename(columns={r'geo\TIME_PERIOD': 'geo'})
data
iso3_map = requests.get("https://country.io/iso3.json").json()
year_to_plot = '2022'
df_filtered = data[(data["indic_il"] == "MED_E") & (data['sex'] == 'T') & (data['age'] == 'TOTAL')]
unit = df_filtered['unit']
geo = df_filtered['geo']
year = df_filtered[year_to_plot]
df = pd.DataFrame({'geo': geo, 'unit': unit, year_to_plot: year})
df["geo"] = df_filtered["geo"].map(iso3_map)
df.dropna(inplace=True)
df = df[(df['unit'] == "EUR") | (df['unit'] == "PPS")]
df
values = df[year_to_plot]
hover_texts = df["geo"] + " - " + df[year_to_plot].astype(str)
hover_texts
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot


init_notebook_mode(connected = True)

data_to_plot = dict(type = 'choropleth',

            locations = df['geo'],


            locationmode = 'ISO-3',


            colorscale = 'Reds',

            text =hover_texts,

            z = values,
            zmin = values.min(),
            zmax = values.max(),

            colorbar = {'title': ('Mediana Dochodu (PPS) w roku ' + year_to_plot)})

layout = dict(geo ={'scope': 'europe'})

choromap = go.Figure(data = [data_to_plot], layout = layout)

#sposób wyświetlenia mapy na google colab
import plotly.io as pio
pio.renderers.default = 'colab'

#iplot(choromap)

choromap.show()

In [23]:
#zad 2
from sklearn.datasets import load_wine

wine = load_wine()
feature_names = wine.feature_names
feature_names
wine_df = pd.DataFrame(np.hstack([wine.data, np.expand_dims(wine.target, axis=1)]), columns=wine.feature_names + ['target'])
print(wine.data.shape)

pca = PCA(n_components=.99)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_1 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_1.shape)

pca = PCA(n_components=.95)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_2 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_2.shape)

pca = PCA(n_components=.90)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_3 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
print(reduced_3.shape)

(178, 13)
(178, 1)
(178, 1)
(178, 1)


In [24]:
#zad 3
pca = PCA(n_components=13)
pca.feature_names_in_ = wine_df.columns[:-1]
reduced_1 = pca.fit_transform(wine_df[wine_df.columns[:-1]], wine_df[wine_df.columns[-1]])
pca.explained_variance_ratio_.round(3)

array([0.998, 0.002, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   ])

In [25]:
#zad 4
X, y = load_wine(return_X_y=True)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 2. Budowa pipeline
pipeline = Pipeline([
    # Standaryzacja: cechy o różnych skalach zostaną wyrównane
    ('scaler', StandardScaler()),

    # Selekcja cech:
    # - RandomForestClassifier oblicza feature_importances_
    # - SelectFromModel odrzuca cechy poniżej mediany importance
    ('feature_sel', SelectFromModel(
        RandomForestClassifier(n_estimators=50, random_state=42),
        threshold='median'
    )),

    # Klasyfikator końcowy: regresja logistyczna
    ('clf', LogisticRegression(max_iter=1000))
])

# 3. Trenowanie całego pipeline
pipeline.fit(X_train, y_train)

# 4. Sprawdzenie liczby cech po selekcji
n_selected = pipeline.named_steps['feature_sel'] \
                    .transform(X_train).shape[1]
print("Liczba cech po selekcji:", n_selected)

# 5. Ocena dokładności na zbiorze walidacyjnym
accuracy = pipeline.score(X_val, y_val)
print("Accuracy na zbiorze walidacyjnym:", accuracy)
feature_names = wine.feature_names  # np. ["sepal length", "sepal width", ...]

# 1. Pobierz krok selekcji z pipeline
selector = pipeline.named_steps['feature_sel']

# 2. Uzyskaj maskę boolean
mask = selector.get_support()
# mask to tablica np. [True, False, True, True] – True = cecha zachowana

# 3. Wypisz nazwy wybranych cech
selected_features = [name for name, keep in zip(feature_names, mask) if keep]
print("Wybrane cechy:", selected_features)

# 4. (Opcjonalnie) Indeksy wybranych cech
import numpy as np
selected_indices = np.where(mask)[0]
print("Indeksy wybranych cech:", selected_indices)

Liczba cech po selekcji: 7
Accuracy na zbiorze walidacyjnym: 1.0
Wybrane cechy: ['alcohol', 'total_phenols', 'flavanoids', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
Indeksy wybranych cech: [ 0  5  6  9 10 11 12]
